In [ ]:
#@title Imports and Installations { vertical-output: true, form-width: "15%" }
from fileinput import filename
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo as pyo
from pytest import param
import pandas as pd
import numpy as np
import os.path
import math
from datetime import datetime
from multiprocessing import Process
#from google.colab import drive
#drive.mount('/content/drive')

Setting up full model

In [ ]:
#for google colab
#@title Optimization Model Code { form-width: "15%" }
class greenAmmoniaProduction:
    def writeDataFile(dataFileName,inputDataset):
        with open('../modelInputs/'+str(dataFileName)+'.dat', 'w') as f:
            
            
            #horizon (time) set
            f.write('set horizon := ')
            for i in range(len(inputDataset["cfWind"])):
                f.write('%d ' % i)
            f.write(';\n\n')

            #ey plants set
            f.write('set eyPlants := ')
            for i in range(len(inputDataset["capexEY"])):
                f.write('%d ' % i)
            f.write(';\n\n')
            
            
            #simplifying writing .dat file with for loop
            paramNames = inputDataset.keys()
            
            #single param index names-for writing correct structure of .dat file
            singleParamIndexNames = ["cfWind","cfSolar","capexEY","fixedOpexEY","variableOpexEY","energyUseEY","stackSize"]
            
            
            for paramName in paramNames:
                if((paramName in ["horizon","eyPlants"])):
                    #skip names as they are sets defined above
                    continue
                elif(paramName in singleParamIndexNames):
                    #writing correct pyomo structure for re generation
                    f.write('param %s := \n' % (paramName))
                    for i in range(len(inputDataset[paramName])):
                        if(i != len(inputDataset[paramName])-1):
                            f.write('%d %f \n' % (i,inputDataset[paramName][i]))
                        else:
                            f.write('%d %f' % (i,inputDataset[paramName][i]))                    
                    f.write(';\n\n')
                else:
                    #all other parameters are single values
                    f.write('param %s := %f; \n' % (paramName,inputDataset[paramName]))
            
            print("Completed data file")
            

    def main(dataFileName,inputDataset,testMode=False):
        """Hourly operations of green ammonia production complex

        Args:
            dataFileName (str): .dat file name which will read in/be created data for model run
            inputDataset (dict): see README for further clarification on what parameters are expected to be inputted into spreadsheet
            testMode (bool): automatically set to false, if true-will delete the .dat input file and output file associated with the dataFileName
        """ 
        #deleting files if test mode is activated
        if(testMode):
            try:
                os.remove(f"../modelInputs/{dataFileName}.dat")
                os.remove(f"../modelOutputs/{dataFileName}.xlsx")
            except:
                print("Test mode activated but one of files already deleted")
        
            
        
        # creating optimization model with pyomo abstract representation
        model = AbstractModel()

        ################### START SETS  ###################
        #timesteps in simulation-based on capacity factor dataset
        model.horizon = RangeSet(0,len(inputDataset["cfSolar"])-1)
        
        #number of unique electroyzer (ey) models to select from
        model.eyPlants = RangeSet(0,len(inputDataset["capexEY"])-1)
        
        ################### END SETS  ###################


        
        ################### START PARAMETERS  ###################
        #ammonia demand over simulation on daily basis (will later be converted to entire time series)
        model.ammoniaDemand = Param()
        
        #respective capacity factor timeseries data for wind and solar site
        model.cfWind = Param(model.horizon)
        model.cfSolar = Param(model.horizon)
        
        #chemical efficiency of storing hydrogen in hydrogen tank
        model.hsStoreEfficiency = Param()
        
        #chemical efficiency of deploying hydrogen from tank
        model.hsDeployEfficiency = Param()

        #energy efficiency of storing energy in battery (e.g. you need to put in 1.2 kW to store 1 kW)
        model.bsStoreEfficiency = Param()
        
        #energy efficiency of deploying energy from battery
        model.bsDeployEfficiency = Param()        
        
        #energy efficiency of fuel cell (converting hydrogen into electricity)
        model.fcDeployEfficiency = Param()
        
        
        #chemical efficiency of using hydrogen in  HB process (e.g.~.18 tons H2 input will output ~1 ton ammonia)
        model.hbHydrogenEfficiency = Param() 
        
        #chemical efficiency of nitrogen into HB process (e.g.~.82 tons N2 input will output ~1 ton ammonia)
        model.hbNitrogenEfficiency = Param() 
                
        #Looking at CAPEX for wind and solar per MW
        model.capexWind = Param()
        model.capexSolar = Param()
        
        #capex for electroysis depending on plant size (to capture economics of scale)
        model.capexEY = Param(model.eyPlants)
        
        #hydrogen storage CAPEX per kg H2
        model.capexHS = Param()

        #battery storage power CAPEX per MWh
        model.capexBSpower = Param()
        
        #battery storage energy CAPEX per MW
        model.capexBSenergy = Param()
        
        #fuel cell CAPEX
        model.capexFC = Param()
        
        #air separation unit CAPEX per MW
        model.capexASU = Param()
        
        #Haber-Bosch ammonia plant  CAPEX per MW
        model.capexHB = Param()
        
        #same outline above for different stages but now looking at fixed OPEX
        model.fixedOpexWind = Param()
        model.fixedOpexSolar = Param()
        model.fixedOpexEY = Param(model.eyPlants)
        model.fixedOpexHS = Param()
        model.fixedOpexBSpower = Param()
        model.fixedOpexFC = Param()
        model.fixedOpexASU = Param()
        model.fixedOpexHB = Param()        

        #Now only looking at variable OPEX for EY as all the other technologies are assumed to have 
        #negligible variable OPEX
        model.variableOpexEY = Param(model.eyPlants)

       
       
        #energy consumption for each EY model type MWh/kg H2
        model.energyUseEY = Param(model.eyPlants)
        
        #energy required to store one kg of hydrogen in high pressure vessel: MWh/kg H2
        model.energyUseHS = Param()
        
        #energy density of hydrogen (around 33.6 kWh per kg of H2)-used for generating electricity if fuel cell deployed
        model.energyDensityFC = Param()
        
        #energy consumption for ASU: MWh/kg N2
        model.energyUseASU = Param()
        
        #energy consumption for Ammonia Plant: MWh/kg NH3
        model.energyUseHB = Param()

        #maximum operating percentage of nameplate capacity for BS
        model.maxCapacityBS = Param()    
        
        #minimum operating percentage of nameplate capacity for BS
        model.minCapacityBS = Param()            
        
        #minimum operating percentage of nameplate capacity for ASU
        model.minCapacityASU = Param()
        
        #minimum operating percentage of nameplate capacity for Ammonia plant        
        model.minCapacityHB = Param()
       
       
       
        #stack size (rated energy for EY-MW) from each EY model type
        model.stackSize = Param(model.eyPlants)
        
        
        #number of years the simulation will run through
        model.simulationLifetime = Param()

        #lifetime of renewable plants
        model.reLifetime = Param()
        
        #lifetime of electroyzers (all treated the same)
        model.eyLifetime = Param()
        
        #lifetime of hydrogen storage
        model.hsLifetime = Param()
        
        #lifetime of battery storage
        model.bsLifetime = Param()

        #fuel cell plant lifetime
        model.fcLifetime = Param()

        #ASU unit lifetime
        model.asuLifetime = Param()
        
        #HB plant lifetime
        model.hbLifetime = Param()

        #ramping rate for ASU
        model.rampingRateASU = Param()

        #ramping rate for HB plant
        model.rampingRateHB = Param()
       
        #WACC or discount rate for plant operations
        model.WACC = Param()
        
        #utilization ratio of plant (not used in model but calculated for total ammonia production)
        model.utilizationRatio = Param()
        ################### END PARAMETERS  ###################
        
        
        
        
        ################### START DECISION VARIABLES  ###################
        #how much wind capacity to build
        model.windCapacity = Var(domain=NonNegativeReals)
        
        #how much solar capacity to build
        model.solarCapacity = Var(domain=NonNegativeReals)
        
        #how many stacks to build of model i for EY (so only integers)
        model.eyCapacity = Var(model.eyPlants, domain = NonNegativeIntegers)
        
        #total battery power storage capacity to build (MWh)
        model.bsPowerCapacity = Var(domain=NonNegativeReals)
        
        #total battery energy storage capacity to build (MW)
        model.bsEnergyCapacity = Var(domain=NonNegativeReals)
       
        #total hydrogen storage capacity to build (kg H2)
        model.hsCapacity = Var(domain=NonNegativeReals)
        
        #fuel cell nameplate capacity (MW)
        model.fcCapacity = Var(domain=NonNegativeReals)
        
        #total ASU capacity to build (kg N2)
        model.asuCapacity = Var(domain=NonNegativeReals)
        
        #total ammonia plant nameplate capacity to build (HB process) (kg NH3)
        model.hbCapacity = Var(domain=NonNegativeReals)
        
        #H2 (kg) to produce from model type i at timestep t
        model.eyGen = Var(model.eyPlants,model.horizon,domain=NonNegativeReals)
        
        #N2 (kg) to produce from ASU at timestep t
        model.asuGen = Var(model.horizon,domain=NonNegativeReals)
        
        #NH3 (kg) to produce from ammonia plant (HB) at timestep t
        model.hbGen = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of hydrogen (kg) to store in tanks at timestep t
        model.hsStore = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of energy (MWh) to store in battery storage at timestep t
        model.bsStore = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of hydrogen (kg) in storage at timestep t which can be used
        model.hsAvail = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of energy (MWh) available in battery at timestep t
        model.bsAvail = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of hydrogen (kg) to deploy to HB process at timestep t
        model.hsDeploy = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of energy (MWh) to release into islanded grid from fuel cell at timestep t
        model.fcDeploy = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of energy (MWh) to release into islanded grid at timestep t
        model.bsDeploy = Var(model.horizon,domain=NonNegativeReals)
        ################### END DECISION VARIABLES    ###################
        
        
        
        ###################     START OBJECTIVE     ###################
        #sum up the CAPEX, fixed OPEX, and variable OPEX costs for each of the 7 components of the green ammonia value chain
        def windCosts(model):
            #sum of capex multiplied by wind capacity built + 
            #operational fixed costs*number of plant years that the OPEX is applied and then factor in time value of money
            return (model.windCapacity*(sum((model.capexWind / (math.pow((1+model.WACC),j*model.reLifetime)) for j in np.arange(math.ceil(model.simulationLifetime/model.reLifetime))))
                                        + sum((model.fixedOpexWind/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))))
             
        def solarCosts(model):
            #sum of capex multiplied by solar capacity built + 
            #operational fixed costs*number of plant years that the OPEX is applied
            return (model.solarCapacity*(sum((model.capexSolar / (math.pow((1+model.WACC),j*model.reLifetime)) for j in np.arange(math.ceil(model.simulationLifetime/model.reLifetime))))
                                         + sum((model.fixedOpexSolar/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))))

        def eyCosts(model):
            # have a temporary fix for running model without annual data for variable OPEX(8760/len*+(model.horizon)))-calculates the scaling factor for full year expenditures
            # stackSize[i]*eyModelCapacity[i] = total MW consumption of ey model i
            # need to then multiply by capexEY (in $/MW) and fixed OPEX ($/MW) including discount factor
            # for variable ($/kg H2) multiply by generation at each hour (e.g. cost of water)
            return (sum(model.stackSize[i]*model.eyCapacity[i]*(sum((model.capexEY[i] / (math.pow((1+model.WACC),j*model.eyLifetime)) for j in np.arange(math.ceil(model.simulationLifetime/model.eyLifetime)))) + 
                    sum((model.fixedOpexEY[i]/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime))))  for i in model.eyPlants) +
                    sum((8760/len(model.horizon))*sum(model.variableOpexEY[i]*model.eyGen[i,t] for i in model.eyPlants)/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))
        
        def hsCosts(model):
            #similar to wind and solar costs-sum up capex for hydrogen storage and then fixed OEPX 
            # however for hsCapacity (in kg) you need to convert to MW as capexHS and fixedOpxHS are in $/MW
            #energyUseHS is in MWh/kg thus multiplying by kg leaves us with MWh on a per hour basis for MW (assuming 1 MW capacity can deploy for 1 MWh)
            return (model.hsCapacity*(sum((model.capexHS / (math.pow((1+model.WACC),j*model.hsLifetime)) for j in np.arange(math.ceil(model.simulationLifetime/model.hsLifetime)))) + 
                    sum((model.fixedOpexHS/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))))
       
        def bsCosts(model):
            #same as hsCosts but dont have to change bsCapacity to MW as already in MWh
            return ((sum((model.bsPowerCapacity*(model.capexBSpower / (math.pow((1+model.WACC),j*model.bsLifetime)))+ model.bsEnergyCapacity*(model.capexBSenergy / (math.pow((1+model.WACC),j*model.bsLifetime)))) for j in np.arange(math.ceil(model.simulationLifetime/model.bsLifetime))) + 
                    model.bsPowerCapacity*sum((model.fixedOpexBSpower/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))))                      

        def fcCosts(model):
            #same as bsCosts however since fuel cell lifetime is so small, will need to have multiple installations
            # over lifetime of the simulation
            return(model.fcCapacity*(sum((model.capexFC / (math.pow((1+model.WACC),j*model.fcLifetime)) for j in np.arange(math.ceil(model.simulationLifetime/model.fcLifetime)))) +
                    sum((model.fixedOpexFC/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))))
        
        
        def asuCosts(model):
            #same structure for hsCosts but changing to unique parameters
            return (model.energyUseASU*model.asuCapacity*(sum((model.capexASU / (math.pow((1+model.WACC),j*model.asuLifetime)) for j in np.arange(math.ceil(model.simulationLifetime/model.asuLifetime)))) + 
                    sum((model.fixedOpexASU/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))))                                                                

        def hbCosts(model):
            #same structure for hsCosts but changing to unique parameters
            return (model.energyUseHB*model.hbCapacity*(sum((model.capexHB / (math.pow((1+model.WACC),j*model.hbLifetime)) for j in np.arange(math.ceil(model.simulationLifetime/model.hbLifetime)))) + 
                    sum((model.fixedOpexHB/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime))))) 
                    
        
        def minCost_rule(model):
            return (windCosts(model) + solarCosts(model) + eyCosts(model) + hsCosts(model) + 
                    bsCosts(model) + fcCosts(model) + asuCosts(model) + hbCosts(model))
        
        model.SystemCost = Objective(rule = minCost_rule, sense = minimize)
        
        ###################       END OBJECTIVE     ###################
    
        ###################       START CONSTRAINTS     ###################
        #meet the ammonia production targets for demand for one year
        def meetAmmoniaDemand(model):
            return(2*sum(model.hbGen[t] for t in model.horizon) == 365*model.ammoniaDemand)
        model.meetAmmoniaDemandConstraint = Constraint(rule=meetAmmoniaDemand)

        #generate enough energy to meet all required islanded components demand
        #Note: the energy usage parameters should encapsulate the energy efficiencies of each of the stages
        def energyDemand(model,t):
            #summing up all the demand from the various energy consumption stages (MWh/kg*kg produced gives us MWh)
            return(2*(sum(model.energyUseEY[i]*model.eyGen[i,t] for i in model.eyPlants) +
                    model.energyUseHS*model.hsStore[t] + (model.bsStore[t]/model.bsStoreEfficiency) +
                    model.energyUseASU*model.asuGen[t] + model.energyUseHB*model.hbGen[t]))
        
        def energyGen(model,t):
            #summing up generation from wind, solar, and battery storage (don't need to include efficiency for bs as 
            # the decision variable BSdeploy is the actual quantity deployed to grid)
            return(2*(model.cfWind[t]*model.windCapacity + model.cfSolar[t]*model.solarCapacity + model.bsDeploy[t] + model.fcDeploy[t]))

        def energyRule(model,t):
            #energy generated should always be equal to or greater than energy demanded
            return(energyDemand(model,t) <= energyGen(model,t))
        
        model.energyConstraint = Constraint(model.horizon,rule=energyRule)
        
        #battery storage operations definition (current energy available is 
        # 30% of rated energy capacity at beginning and equal to previous available amount + new energy stored - (energy deployed + energy required to deploy it))
        def bsAvailEnergyRule(model,t):
            if t == 0:
                return(model.bsAvail[0] == .3*model.bsEnergyCapacity)
            else:
                return(model.bsAvail[t] == model.bsAvail[t-1] + 2*model.bsStore[t-1] - 2*(model.bsDeploy[t-1])/model.bsDeployEfficiency)
        model.bsAvailEnergyDefConstraint = Constraint(model.horizon,rule=bsAvailEnergyRule)

        #max available energy you can have in storage is storage capacity multiplied by max operating percentage (usually 95%)
        def bsAvailUpperBoundRule(model,t):
            return(model.bsAvail[t] <= model.maxCapacityBS*model.bsEnergyCapacity)
        model.bsAvailUpperBoundConstraint = Constraint(model.horizon,rule=bsAvailUpperBoundRule)

        #max state of charge of battery is storage capacity multiplied by min operating percentage
        def bsAvailLowerBoundRule(model,t):
            return(model.bsAvail[t] >= model.minCapacityBS*model.bsEnergyCapacity)
        model.bsAvailLowerBoundConstraint = Constraint(model.horizon,rule=bsAvailLowerBoundRule)
        
        #max amount of energy you can store in bs is max capacity - current energy charge at start - energy you deploy in hour
        #for simplicity, I assume you can not store energy in the hour and deploy it within the same hour-this would require finer resolution then hourly capacity factors
        def bsStoreEnergyUpperBoundRule(model,t):
            return(2*model.bsStore[t] <= model.bsEnergyCapacity - model.bsAvail[t])
        model.bsStoreEnergyUpperBoundConstraint = Constraint(model.horizon,rule=bsStoreEnergyUpperBoundRule)

        #max amount of power you can store or deploy in bs in an hour
        def bsPowerUpperBoundRule(model,t):
            return(model.bsStore[t] + (model.bsDeploy[t]/model.bsDeployEfficiency) <= model.bsPowerCapacity)
        model.bsPowerUpperBoundConstraint = Constraint(model.horizon,rule=bsPowerUpperBoundRule)
        

        #maximum energy to power ratio of 10 hours
        def bsPowerEnergyRatioMaxRule(model):
            return(model.bsEnergyCapacity <= 10*model.bsPowerCapacity)
        model.bsPowerEnergyRatioMaxConstraint = Constraint(rule=bsPowerEnergyRatioMaxRule)


        #energy deployed (and thus required) from storage must be less than energy available
        def bsDeployUpperBoundRule(model,t):
            return(2*(model.bsDeploy[t]/model.bsDeployEfficiency) <= model.bsAvail[t])
                
        model.bsDeployUpperBoundConstraint = Constraint(model.horizon,rule=bsDeployUpperBoundRule)

        #fuel cell operations definition (must be less than capacity built)
        def fcOperationsRule(model,t):
            return(model.fcDeploy[t] <= model.fcCapacity)
        model.fcDeployOperationsConstraint = Constraint(model.horizon,rule=fcOperationsRule)


        #fuel cell hydrogen deployment definition (need to convert kg to MWh and then factor in efficiency of fuel cell)
        def fcDeployUpperBoundRule(model,t):
            return(2*model.fcDeploy[t] <= model.fcDeployEfficiency*model.energyDensityFC*model.hsAvail[t])
        model.fcDeployUpperBoundConstraint = Constraint(model.horizon,rule=fcDeployUpperBoundRule)
        


        #hydrogen storage operations definition (current hydrogen available is zero at beginning and equal to previous available amount + new hydrogen stored - (hydrogen deployed and hydrogen required to deploy it for fuel cell and HB process))
        def hsAvailEnergyRule(model,t):
            if t == 0:
                return(model.hsAvail[0] == 0)
            else:
                return(model.hsAvail[t] == model.hsAvail[t-1] + 2*model.hsStore[t-1] - 2*(model.hsDeploy[t-1]/model.hsDeployEfficiency) - 2*(model.fcDeploy[t-1]/(model.fcDeployEfficiency*model.energyDensityFC)))
        model.hsAvailEnergyDefConstraint = Constraint(model.horizon,rule=hsAvailEnergyRule)

        #max available hydrogen in storage is storage capacity
        def hsAvailUpperBoundRule(model,t):
            return(model.hsAvail[t] <= model.hsCapacity)
        model.hsAvailUpperBoundConstraint = Constraint(model.horizon,rule=hsAvailUpperBoundRule)
        
        #max amount of hydrogen you can store in hs is remaining space available (diff of hydrogen capacity - hydrogen available)
        def hsStoreUpperBoundRule(model,t):
            return(2*model.hsStore[t] <= model.hsCapacity - model.hsAvail[t])
        model.hsStoreUpperBoundConstraint = Constraint(model.horizon,rule=hsStoreUpperBoundRule)
        
        #hydrogen deployed (and also required to deploy) from storage must be less than hydrogen available
        def hsDeployUpperBoundRule(model,t):
            return(2*(model.hsDeploy[t]/model.hsDeployEfficiency) <= model.hsAvail[t])
        model.hsDeployUpperBoundConstraint = Constraint(model.horizon,rule=hsDeployUpperBoundRule)
                
        #amount of hydrogen to store (and lost in storing) must be less than or equal to hydrogen generated from all EY stacks
        def hsStoreGenUpperBoundRule(model,t):
            return((model.hsStore[t]/model.hsStoreEfficiency) <= 2*sum(model.eyGen[i,t] for i in model.eyPlants))
        model.hsStoreGenUpperBoundConstraint = Constraint(model.horizon,rule=hsStoreGenUpperBoundRule)
        
        #hydrogen production (in kg) from all the stacks in model i must be less than total number built*output per unit (have to convert stackSize (MW) to kg (and thus divide by energyUsage (MWh/kg)))
        def eyGenUpperBoundRule(model,i,t):
            return(model.eyGen[i,t] <= (model.stackSize[i]/model.energyUseEY[i])*model.eyCapacity[i])
        model.eyGenUpperBoundConstraint = Constraint(model.eyPlants,model.horizon,rule=eyGenUpperBoundRule)
        
        #ASU production can't exceed max capacity built (kg)
        def asuGenUpperBoundRule(model,t):
            return(model.asuGen[t] <= model.asuCapacity)
        model.asuGenUpperBoundConstraint = Constraint(model.horizon,rule=asuGenUpperBoundRule)

        #Ammonia plant production (HB) can't exceed max capacity built (kg)
        def hbGenUpperBoundRule(model,t):
            return(model.hbGen[t] <= model.hbCapacity)
        model.hbGenUpperBoundConstraint = Constraint(model.horizon,rule=hbGenUpperBoundRule)        
        
        #ramp up constraint for ASU
        def asuGenRampRateUpRule(model,t):
            if(t==0):
                return(model.asuGen[t] <= model.asuCapacity)
            else:
                return((model.asuGen[t]-model.asuGen[t-1]) <= 2*model.rampingRateASU*model.asuCapacity)
        model.asuGenRampRateUpConstraint = Constraint(model.horizon,rule=asuGenRampRateUpRule)        

        #ramp down constraint for ASU
        def asuGenRampRateDownRule(model,t):
            if(t==0):
                return(model.asuGen[t] <= model.asuCapacity)
            else:
                return((model.asuGen[t-1]-model.asuGen[t]) <= 2*model.rampingRateASU*model.asuCapacity)
        model.asuGenRampRateDownConstraint = Constraint(model.horizon,rule=asuGenRampRateDownRule)   


        #ramping constraint up for HB plant
        def hbGenRampRateUp(model,t):
            if(t==0):
                return(model.hbGen[t] <= model.hbCapacity)
            else:
                return((model.hbGen[t]-model.hbGen[t-1]) <= 2*model.rampingRateHB*model.hbCapacity)
        model.hbGenRampRateUp = Constraint(model.horizon,rule=hbGenRampRateUp)                
        
        
        #ramping constraint down for HB plant
        def hbGenRampRateDown(model,t):
            if(t==0):
                return(model.hbGen[t] <= model.hbCapacity)
            else:
                return((model.hbGen[t-1]-model.hbGen[t]) <= 2*model.rampingRateHB*model.hbCapacity)
        model.hbGenRampRateDown = Constraint(model.horizon,rule=hbGenRampRateDown)      
        

        #ASU plant production can not fall below its minimum hourly production capacity
        def asuGenLowerBoundRule(model,t):
            return(model.asuGen[t] >= model.minCapacityASU*model.asuCapacity)
        model.asuGenLowerBoundConstraint = Constraint(model.horizon,rule=asuGenLowerBoundRule)

        #Ammonia plant production (HB) can't fall below its minimum hourly production capacity
        def hbGenLowerBoundRule(model,t):
            return(model.hbGen[t] >= model.minCapacityHB*model.hbCapacity)
        model.hbGenLowerBoundConstraint = Constraint(model.horizon,rule=hbGenLowerBoundRule)        
        
        #hydrogen input to the ammonia plant (hydrogen deployed directly from eyGeneration and then hydrogen deployed from storage)
        # must be equal to the required hydrogen input stoichiometric ratio
        def hbHydrogenInputRule(model,t):
            return((1/model.hbHydrogenEfficiency)*(sum(model.eyGen[i,t]  for i in model.eyPlants)-model.hsStore[t]/model.hsStoreEfficiency + model.hsDeploy[t]) == model.hbGen[t])
        model.hbGenHydrogenInputConstraint = Constraint(model.horizon,rule=hbHydrogenInputRule)        
 
        #nitrogen input to the ammonia plant must be equal to the required nitrogen input stoichiometric ratio 
        def hbNitrogenInputRule(model,t):
            return((1/model.hbNitrogenEfficiency)*(model.asuGen[t]) == model.hbGen[t])
        model.hbGenNitrogenInputConstraint = Constraint(model.horizon,rule=hbNitrogenInputRule)        
         
        ###################       END   CONSTRAINTS     ###################
    


        ###################          WRITING DATA       ###################
        if(os.path.isfile(f"../dataInputs/{dataFileName}.dat")):
            print(f"Data file {dataFileName} already exists!\nSkipping creating .dat file")
        else:
            #print(f"Data file {dataFileName} does not exist.\nCreating .dat file")
            greenAmmoniaProduction.writeDataFile(dataFileName,inputDataset)
        
        # load in data for the system
        data = DataPortal()
        data.load(filename=f"../modelInputs/{dataFileName}.dat", model=model)
        instance = model.create_instance(data)
                
        
        
        solver = SolverFactory('glpk')
        result = solver.solve(instance)
        #instance.display()
        
        
        #setting up structure in order to get out decision variables (and objective) and save in correct excel format
        singleDecisionVariables = ["windCapacity","solarCapacity","bsPowerCapacity","bsEnergyCapacity",
                                        "hsCapacity","fcCapacity","asuCapacity","hbCapacity", "totalSystemCost",
                                        "LCOA","windCosts","solarCosts","eyCosts","hsCosts",
                                        "bsPowerCosts","bsEnergyCosts","fcCosts","asuCosts","hbCosts","windCapexCosts",
                                        "windOpexCosts","solarCapexCosts","solarOpexCosts",
                                        "eyCapexCosts","eyOpexCosts","hsCapexCosts","hsOpexCosts",
                                        "bsPowerCapexCosts", "bsPowerOpexCosts","fcCapexCosts","fcOpexCosts",
                                        "asuCapexCosts","asuOpexCosts","hbCapexCosts","hbOpexCosts"]
        
        
        
        #included wind and solar generation for simplifying data analysis and timestep
        hourlyDecisionVariables = ["windGen","solarGen","asuGen","hbGen","hsStore",
                                   "bsStore","hsAvail","bsAvail", "hsDeploy","bsDeploy","fcDeploy","timestep"]
        
        #eyDecisionVariables =  ["eyCapacity"]
        
        #eyHourlyDecisionVariables = ["eyGen"]
        

        singleDvDataset = pd.DataFrame(0.0, index=np.arange(1), columns=singleDecisionVariables)

        hourlyDvDataset = pd.DataFrame(0.0, index=np.arange(len(inputDataset["cfSolar"])), columns=hourlyDecisionVariables)
        
        eySingleDvDataset = pd.DataFrame(0.0, index=np.arange(len(inputDataset["capexEY"])), columns=np.arange(len(inputDataset["capexEY"]))) 
        
        eyHourlyDvDataset = pd.DataFrame(0.0, index=np.arange(len(inputDataset["cfSolar"])), columns=np.arange(len(inputDataset["capexEY"]))) 
        
        #assigning single values to df
        singleDvDataset["windCapacity"][0] = instance.windCapacity.value
        singleDvDataset["solarCapacity"][0] = instance.solarCapacity.value
            
        singleDvDataset["bsPowerCapacity"][0] = instance.bsPowerCapacity.value
        singleDvDataset["bsEnergyCapacity"][0] = instance.bsEnergyCapacity.value                        
        singleDvDataset["hsCapacity"][0] = instance.hsCapacity.value
        singleDvDataset["fcCapacity"][0] = instance.fcCapacity.value                    
        singleDvDataset["asuCapacity"][0] = instance.asuCapacity.value            
        singleDvDataset["hbCapacity"][0] = instance.hbCapacity.value            
        singleDvDataset["totalSystemCost"][0] = value(instance.SystemCost)
        
        #still assigning single values to df however looking at LCOA and various segments contributing
        #factor in 2 times as only looking at half a year of production
        totalAmmoniaProduction = instance.utilizationRatio*sum((365*inputDataset["ammoniaDemand"])/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))
        #totalAmmoniaProduction = instance.utilizationRatio*sum((inputDataset["ammoniaDemand"]*8760/len(instance.horizon))/(math.pow((1+0),t)) for t in np.arange(instance.simulationLifetime))
        
        
        singleDvDataset["LCOA"][0] = value(instance.SystemCost)/totalAmmoniaProduction
        
        singleDvDataset["windCosts"][0] = (instance.windCapacity.value*(sum((instance.capexWind / (math.pow((1+instance.WACC),j*instance.reLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.reLifetime))))
                                        + sum((instance.fixedOpexWind/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalAmmoniaProduction        
        singleDvDataset["windCapexCosts"] = (instance.windCapacity.value*(sum((instance.capexWind / (math.pow((1+instance.WACC),j*instance.reLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.reLifetime))))))/totalAmmoniaProduction
        singleDvDataset["windOpexCosts"] = (instance.windCapacity.value*(sum((instance.fixedOpexWind/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalAmmoniaProduction
        
        
        
        singleDvDataset["solarCosts"][0] = (instance.solarCapacity.value*(sum((instance.capexSolar / (math.pow((1+instance.WACC),j*instance.reLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.reLifetime))))
                                        + sum((instance.fixedOpexSolar/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalAmmoniaProduction        
        singleDvDataset["solarCapexCosts"] = (instance.solarCapacity.value*(sum((instance.capexSolar / (math.pow((1+instance.WACC),j*instance.reLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.reLifetime))))))/totalAmmoniaProduction
        singleDvDataset["solarOpexCosts"] = (instance.solarCapacity.value*(sum((instance.fixedOpexSolar/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalAmmoniaProduction
        
        
        
        singleDvDataset["eyCosts"][0] = (sum(instance.stackSize[i]*instance.eyCapacity[i].value*(sum((instance.capexEY[i] / (math.pow((1+instance.WACC),j*instance.eyLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.eyLifetime)))) + 
                    sum((instance.fixedOpexEY[i]/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))))  for i in instance.eyPlants) +
                    sum((8760/len(instance.horizon))*sum(instance.variableOpexEY[i]*instance.eyGen[i,t].value for i in instance.eyPlants)/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))/totalAmmoniaProduction
        singleDvDataset["eyCapexCosts"] = (sum(instance.stackSize[i]*instance.eyCapacity[i].value*(sum((instance.capexEY[i] / (math.pow((1+instance.WACC),j*instance.eyLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.eyLifetime)))))  for i in instance.eyPlants))/totalAmmoniaProduction
        singleDvDataset["eyOpexCosts"] = (sum(instance.stackSize[i]*instance.eyCapacity[i].value*sum((instance.fixedOpexEY[i]/(math.pow((1+value(instance.WACC)),t))) for t in np.arange(instance.simulationLifetime)) for i in instance.eyPlants) + sum((8760/len(instance.horizon))*sum(instance.variableOpexEY[i]*instance.eyGen[i,t].value for i in instance.eyPlants)/(math.pow((1+value(instance.WACC)),t)) for t in np.arange(instance.simulationLifetime)))/totalAmmoniaProduction
        
        
        
        singleDvDataset["hsCosts"][0] = (instance.hsCapacity.value*(sum((instance.capexHS / (math.pow((1+instance.WACC),j*instance.hsLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.hsLifetime)))) + 
                    sum((instance.fixedOpexHS/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalAmmoniaProduction
        singleDvDataset["hsCapexCosts"] = (instance.hsCapacity.value*(sum((instance.capexHS / (math.pow((1+instance.WACC),j*instance.hsLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.hsLifetime))))))/totalAmmoniaProduction
        singleDvDataset["hsOpexCosts"] = (instance.hsCapacity.value*sum((instance.fixedOpexHS/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))))/totalAmmoniaProduction
        
        
        singleDvDataset["bsPowerCosts"][0] = (sum(instance.bsPowerCapacity.value*(instance.capexBSpower / (math.pow((1+instance.WACC),j*instance.bsLifetime)))for j in np.arange(math.ceil(instance.simulationLifetime/instance.bsLifetime))) +
                                              instance.bsPowerCapacity.value*sum((instance.fixedOpexBSpower/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))))/totalAmmoniaProduction                      
        singleDvDataset["bsEnergyCosts"][0] = (sum(instance.bsEnergyCapacity.value*(instance.capexBSenergy / (math.pow((1+instance.WACC),j*instance.bsLifetime))) for j in np.arange(math.ceil(instance.simulationLifetime/instance.bsLifetime))))/totalAmmoniaProduction        
        singleDvDataset["bsPowerCapexCosts"] = (sum(instance.bsPowerCapacity.value*(instance.capexBSpower / (math.pow((1+instance.WACC),j*instance.bsLifetime)))for j in np.arange(math.ceil(instance.simulationLifetime/instance.bsLifetime))))/totalAmmoniaProduction
        singleDvDataset["bsPowerOpexCosts"] = (instance.bsPowerCapacity.value*sum((instance.fixedOpexBSpower/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))))/totalAmmoniaProduction
        
        
        singleDvDataset["fcCosts"][0] = (instance.fcCapacity.value*(sum((instance.capexFC / (math.pow((1+instance.WACC),j*instance.fcLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.fcLifetime)))) +
                    sum((instance.fixedOpexFC/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalAmmoniaProduction                      
        singleDvDataset["fcCapexCosts"] = (instance.fcCapacity.value*(sum((instance.capexFC / (math.pow((1+instance.WACC),j*instance.fcLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.fcLifetime))))))/totalAmmoniaProduction
        singleDvDataset["fcOpexCosts"] = (instance.fcCapacity.value*sum((instance.fixedOpexFC/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))))/totalAmmoniaProduction


        
        singleDvDataset["asuCosts"][0] = (instance.energyUseASU*instance.asuCapacity.value*(sum((instance.capexASU / (math.pow((1+instance.WACC),j*instance.asuLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.asuLifetime)))) + 
                    sum((instance.fixedOpexASU/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))                                                                /totalAmmoniaProduction                                                                
        singleDvDataset["asuCapexCosts"] = (instance.energyUseASU*instance.asuCapacity.value*(sum((instance.capexASU / (math.pow((1+instance.WACC),j*instance.asuLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.asuLifetime))))))/totalAmmoniaProduction
        singleDvDataset["asuOpexCosts"] = (instance.energyUseASU*instance.asuCapacity.value*sum((instance.fixedOpexASU/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))))/totalAmmoniaProduction
        
        
        singleDvDataset["hbCosts"][0] = (instance.energyUseHB*instance.hbCapacity.value*(sum((instance.capexHB / (math.pow((1+instance.WACC),j*instance.hbLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.hbLifetime)))) + 
                    sum((instance.fixedOpexHB/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalAmmoniaProduction 
        singleDvDataset["hbCapexCosts"] = (instance.energyUseHB*instance.hbCapacity.value*(sum((instance.capexHB / (math.pow((1+instance.WACC),j*instance.hbLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.hbLifetime))))))/totalAmmoniaProduction
        singleDvDataset["hbOpexCosts"] = (instance.energyUseHB*instance.hbCapacity.value*sum(instance.fixedOpexHB/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))/totalAmmoniaProduction
        
        #including load factors for EY,ASU, and HB
        singleDvDataset["asuLoadFactor"] =  sum(instance.asuGen[hour].value for hour in np.arange(len(inputDataset["cfSolar"])))/(len(inputDataset["cfSolar"])*instance.asuCapacity.value)
        singleDvDataset["hbLoadFactor"] =  sum(instance.hbGen[hour].value for hour in np.arange(len(inputDataset["cfSolar"])))/(len(inputDataset["cfSolar"])*instance.hbCapacity.value)
        
        
        
        
        #assigning hourly values to dfs
        for hour in np.arange(len(inputDataset["cfSolar"])):
            hourlyDvDataset["windGen"][hour] = singleDvDataset["windCapacity"][0]*inputDataset["cfWind"][hour]
            hourlyDvDataset["solarGen"][hour] = singleDvDataset["solarCapacity"][0]*inputDataset["cfSolar"][hour]
            hourlyDvDataset["asuGen"][hour] = instance.asuGen[hour].value
            hourlyDvDataset["hbGen"][hour] = instance.hbGen[hour].value    
            hourlyDvDataset["hsStore"][hour] = instance.hsStore[hour].value    
            hourlyDvDataset["bsStore"][hour] = instance.bsStore[hour].value    
            hourlyDvDataset["hsAvail"][hour] = instance.hsAvail[hour].value    
            hourlyDvDataset["bsAvail"][hour] = instance.bsAvail[hour].value    
            hourlyDvDataset["hsDeploy"][hour] = instance.hsDeploy[hour].value    
            hourlyDvDataset["bsDeploy"][hour] = instance.bsDeploy[hour].value
            hourlyDvDataset["fcDeploy"][hour] = instance.fcDeploy[hour].value
            
            #for later data analysis
            hourlyDvDataset["timestep"][hour] = hour + 1   

        #assigning single dvs for ey types
        eySingleDvDataset = pd.DataFrame(columns=np.arange(len(inputDataset["capexEY"])), index=range(2))
        for eyUnit in np.arange(len(inputDataset["capexEY"])):
            #looking at capacity and load factor
            eySingleDvDataset[eyUnit][0] = instance.stackSize[eyUnit]*instance.eyCapacity[eyUnit].value
            if(instance.eyCapacity[eyUnit].value != 0):
                eySingleDvDataset[eyUnit][1] = sum(instance.eyGen[eyUnit,hour].value for hour in np.arange(len(instance.horizon)))/(len(instance.horizon)*(1/instance.energyUseEY[eyUnit])*instance.stackSize[eyUnit]*instance.eyCapacity[eyUnit].value)
            else:
                eySingleDvDataset[eyUnit][1] = 0

        #changing index names
        eySingleDvDataset = eySingleDvDataset.rename(columns={'0': 'Small EY', "1": "Large EY"}, index={'0': 'Capacity (MW)','1': 'Capacity factor (%)'})

        #assigning hourly dvs for each ey unit
        for eyUnit in np.arange(len(inputDataset["capexEY"])):
            for hour in np.arange(len(inputDataset["cfSolar"])):
                eyHourlyDvDataset[eyUnit][hour] = instance.eyGen[eyUnit,hour].value          
        
        
        #now saving 4 datasets to different sheets in same excel file
        excelOutputFileName = f"../modelOutputs/{dataFileName}.xlsx"
        with pd.ExcelWriter(excelOutputFileName) as writer:  
            singleDvDataset.to_excel(writer,sheet_name='singleValueDvs') 
            
            hourlyDvDataset.to_excel(writer,sheet_name='hourlyValueDvs') 
            

            eySingleDvDataset.to_excel(writer,sheet_name='singleEyValueDvs') 
            
            eyHourlyDvDataset.to_excel(writer,sheet_name='hourlyEyValueDvs') 
        
        print(f"Model output results saved to {excelOutputFileName}")

Now setting up model run

In [ ]:
#@title Single Run { form-width: "15%" }

# # # # START OF SPECIFIC PARAMETER CHANGES # # # # 

#since we are running model on bihour data, we take two runs
#yearPart = 0 looks at half of year data, 1 is other half (is bi hour to get full year sample)
yearPart = 1

#year: 0-2017, 1-2018, 2-2019
year = 2


siteName = "centralWest"

# # # # END OF SPECIFIC PARAMETER CHANGES # # # # 


yearList = ["2017","2018","2019"]
testMode = False
startDay = year*365
endDay = (year+1)*365
transmissionLosses = .02

#dont include the .dat ending or path location
datFileName = f"{siteName}_part{yearPart}_{yearList[year]}"

siteLocation = f"{siteName}Site"


pdSingleParamDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='systemSettings')

paramNames = pdSingleParamDataset["ParamName"]
paramValues = pdSingleParamDataset["Value"]

#creating input dataset for single value Params
inputDataset = {}

#adding wind and solar generation capacity factors
windCFDataset = pd.read_excel(f"../dataInputs/sites/{siteLocation}/reData.xlsx",usecols=["cfWind"])
solarCFDataset = pd.read_excel(f"../dataInputs/sites/{siteLocation}/reData.xlsx",usecols=["cfSolar"])


#getting out wind and solar every other one data
fullYearWind = np.array(windCFDataset["cfWind"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)
inputDataset["cfWind"] = fullYearWind[yearPart::2]


fullYearSolar = np.array(solarCFDataset["cfSolar"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)
inputDataset["cfSolar"] = fullYearSolar[yearPart::2]

#adding single param value data
for paramName,paramValue in zip(paramNames,paramValues):
    inputDataset[paramName] = paramValue


pdEYDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='eyUnitSettings')
#adding EY unit data
for paramName in pdEYDataset.columns:
    if(paramName == "Name"):
        #skip assigning the Name column as the model uses integer indices
        continue
    #create empty array inside dict
    inputDataset[paramName] = np.zeros(len(pdEYDataset["capexEY"]))
    for index in np.arange(0,len(pdEYDataset["capexEY"])):
        inputDataset[paramName][index] = pdEYDataset[paramName][index]


# for tracking elapsed time
startRun = datetime.now()
start_time = startRun.strftime("%H:%M:%S")
print("Run started:", start_time)
print(f"Site: {siteName}")
print(f"Year: {yearList[year]}")
print(f"Part: {yearPart}")

greenAmmoniaProduction.main(datFileName,inputDataset,testMode)


#getting end time
endRun = datetime.now()
end_time = endRun.strftime("%H:%M:%S")
print("Run over:", end_time)

#printing total model runtime
print("Total model time took: ",str(endRun-startRun))

In [ ]:
#@title Read in input dataset for batch dataset{ form-width: "15%" }
#excat same structure as single run, but not running the model-saving it for next cell

#changing to correct directory to run model
os.chdir('/content/drive/MyDrive/KAUST!!/researchProj/ammoniaProduction/src')

# # # # START OF SPECIFIC PARAMETER CHANGES # # # # 

#since we are running model on bihour data, we take two runs
#yearPart = 0 looks at half of year data, 1 is other half (is bi hour to get full year sample)
yearPart = 0

#year: 0-2017, 1-2018, 2-2019
year = 1


siteName = "centralWest"

# # # # END OF SPECIFIC PARAMETER CHANGES # # # # 


yearList = ["2017","2018","2019"]
testMode = False
startDay = year*365
endDay = (year+1)*365
transmissionLosses = .02

#dont include the .dat ending or path location
datFileName = f"{siteName}_part{yearPart}_{yearList[year]}"

siteLocation = f"{siteName}Site"


pdSingleParamDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='systemSettings')

paramNames = pdSingleParamDataset["ParamName"]
paramValues = pdSingleParamDataset["Value"]

#creating input dataset for single value Params
inputDataset = {}

#adding wind and solar generation capacity factors
windCFDataset = pd.read_excel(f"../dataInputs/sites/{siteLocation}/reData.xlsx",usecols=["cfWind"])
solarCFDataset = pd.read_excel(f"../dataInputs/sites/{siteLocation}/reData.xlsx",usecols=["cfSolar"])


#getting out wind and solar every other one data
fullYearWind = np.array(windCFDataset["cfWind"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)
inputDataset["cfWind"] = fullYearWind[yearPart::2]


fullYearSolar = np.array(solarCFDataset["cfSolar"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)
inputDataset["cfSolar"] = fullYearSolar[yearPart::2]

#adding single param value data
for paramName,paramValue in zip(paramNames,paramValues):
    inputDataset[paramName] = paramValue


pdEYDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='eyUnitSettings')
#adding EY unit data
for paramName in pdEYDataset.columns:
    if(paramName == "Name"):
        #skip assigning the Name column as the model uses integer indices
        continue
    #create empty array inside dict
    inputDataset[paramName] = np.zeros(len(pdEYDataset["capexEY"]))
    for index in np.arange(0,len(pdEYDataset["capexEY"])):
        inputDataset[paramName][index] = pdEYDataset[paramName][index]

In [ ]:
#@title Single Parameter Batch Run { form-width: "15%" }


def runModel(fileName,inputDataset):
  #running model with specific parameter input
  greenAmmoniaProduction.main(fileName,inputDataset,False)

#identifying which parameter we want to run a batch run on
parameterTested = "capexWind"
initialValue = 956000
#range to test
#percentRange = [0.5,0.6,0.7,0.8,0.9]
percentRange = [1.1,1.2,1.3,1.4,1.5]

parameterData =  initialValue*np.array(percentRange)

parameterFileList = []
for percent in percentRange:
  parameterFileList.append(f"{parameterTested}_{percent}")

# for tracking elapsed time
startRun = datetime.now()
start_time = startRun.strftime("%H:%M:%S")
print("Batch run started:", start_time)

processes = []

#printing out file information
print("############## Parameter files running ##############")

print(f"Year part: {yearPart}")

for parameterFile in parameterFileList:
  print(parameterFile)

print("##############                         ##############")



# creating processes
for w in range(len(parameterData)):

    #creating unique file structure for name
    fileName = f"{siteLocation}_year{yearList[year]}_part_{yearPart}_{parameterFileList[w]}"

    if(parameterTested == "capexEY"):
      #have multiple electroyzers so need to select large one
      inputDataset[parameterTested][1] = parameterData[w]
    else:
      inputDataset[parameterTested] = parameterData[w]

    #changing OPEX costs
    if(parameterTested == "capexSolar"):
      inputDataset["fixedOpexSolar"] = parameterData[w]*.02
    elif(parameterTested == "capexWind"):
      inputDataset["fixedOpexWind"] = parameterData[w]*.02
    elif(parameterTested == "capexBSpower"):
      inputDataset["fixedOpexBSpower"] = inputDataset["fixedOpexBSpower"]*percentRange[w]
    elif(parameterTested == "capexEY"):
      inputDataset["fixedOpexEY"][1] = parameterData[w]*.02

    p = Process(target=runModel, args=(fileName,inputDataset))
    processes.append(p)
    p.start()

# completing process
for p in processes:
    p.join()


####### RESETING VALUES #######

#adding single param value data
for paramName,paramValue in zip(paramNames,paramValues):
    inputDataset[paramName] = paramValue


pdEYDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='eyUnitSettings')
#adding EY unit data
for paramName in pdEYDataset.columns:
    if(paramName == "Name"):
        #skip assigning the Name column as the model uses integer indices
        continue
    #create empty array inside dict
    inputDataset[paramName] = np.zeros(len(pdEYDataset["capexEY"]))
    for index in np.arange(0,len(pdEYDataset["capexEY"])):
        inputDataset[paramName][index] = pdEYDataset[paramName][index]

####### RESETING VALUES #######


#getting end time
endRun = datetime.now()
end_time = endRun.strftime("%H:%M:%S")
print("Run over:", end_time)


#printing total model runtime
print("Total model time took: ",str(endRun-startRun))



Batch run started: 20:21:57
############## Parameter files running ##############
Year part: 0
capexWind_1.1
capexWind_1.2
capexWind_1.3
capexWind_1.4
capexWind_1.5
##############                         ##############
Completed data file
Completed data file
Completed data file
Completed data fileCompleted data file

Model output results saved to ../modelOutputs/centralWestSite_year2018_part_0_capexWind_1.4.xlsx
Model output results saved to ../modelOutputs/centralWestSite_year2018_part_0_capexWind_1.1.xlsx
Model output results saved to ../modelOutputs/centralWestSite_year2018_part_0_capexWind_1.5.xlsx
Model output results saved to ../modelOutputs/centralWestSite_year2018_part_0_capexWind_1.2.xlsx


In [ ]:
#@title Test Speed { form-width: "15%" }


def runModel(fileName,inputDataset):
  #running model with specific parameter input
  greenAmmoniaProduction.main(fileName,inputDataset,True)


#identifying which parameter we want to run a batch run on
parameterTested = "ammoniaDemand"
parameterData = [10_000,100_000,1_000_000]
parameterFileList = ["ammoniaD_10_000","ammoniaD_100_000","ammoniaD_1_000_000"]

# for tracking elapsed time
startRun = datetime.now()
start_time = startRun.strftime("%H:%M:%S")
print("Batch run started:", start_time)

processes = []

# creating processes
for w in range(len(parameterData)):
    fileName = parameterFileList[w]
    inputDataset[parameterTested] = parameterData[w]
    greenAmmoniaProduction.main(fileName,inputDataset,True)





#getting end time
endRun = datetime.now()
end_time = endRun.strftime("%H:%M:%S")
print("Run over:", end_time)

#printing total model runtime
print("Total model time took: ",str(endRun-startRun))

